# homuncu_loc image viewer

In [1]:
import napari
from skimage import io
import btrack
import os
from tqdm.auto import tqdm

def find_files_with_basename(target_basename, start_dir='.'):
    """
    Recursively search for files with a given basename in a directory tree.
    
    :param target_basename: The basename to search for.
    :param start_dir: The directory to start the search from. Default is current directory.
    :return: A list of full paths to files matching the given basename.
    """
    matching_files = []
    
    for dirpath, dirnames, filenames in tqdm(os.walk(start_dir), desc = 'Searching start directory for filename'):
        for fname in filenames:
            if os.path.basename(fname) == target_basename:
                full_path = os.path.join(dirpath, fname)
                matching_files.append(full_path)
                
    return matching_files

def find_tif_files(start_dir='.'):
    """
    Recursively search for files with '.tif' extension in a directory tree.
    
    :param start_dir: The directory to start the search from. Default is current directory.
    :return: A list of full paths to files with '.tif' extension.
    """
    matching_files = []
    
    for dirpath, dirnames, filenames in tqdm(os.walk(start_dir), desc='Searching for .tif files'):
        for fname in filenames:
            if fname.endswith('.tif'):
                full_path = os.path.join(dirpath, fname)
                matching_files.append(full_path)
                
    return matching_files

#### Define image fn 

In [7]:
image_fn = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/Job_Mtb area/run3/23-03-002/20230801_20X_23-03-002A5_DAPI_NKX201_PDPN_ZO-1_Multichannel Z-Stack_20230801_1441.tif'

#### Or search for image basename in directory

In [139]:
image_basename = '20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1313.tif'
start_directory = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/'

In [140]:
image_fn_results = find_files_with_basename(image_basename, start_directory)
print(image_fn_results)

Searching start directory for filename: 0it [00:00, ?it/s]

['/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/Job_Mtb area/run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1313.tif']


In [141]:
image_fn = image_fn_results[0]

In [76]:
%%time
image = io.imread(image_fn)
image.shape

CPU times: user 1.21 s, sys: 12.3 s, total: 13.5 s
Wall time: 3min 49s


(61, 2304, 2304, 4)

#### Now do the same for the tracking file

In [142]:
tracks_basename = image_basename.replace('.tif','_z_tracks_masks.h5') #'20x_21-12-029B_A12346_Multichannel Z-Stack_20220819_295_z_tracks_masks.h5'
tracks_fn_results = find_files_with_basename(tracks_basename, start_directory)
if len(tracks_fn_results) > 1:
        raise ValueError("Found more than one matching file. Please ensure only one target file exists or refine your search criteria.")
print(tracks_fn_results)

Searching start directory for filename: 0it [00:00, ?it/s]

['/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/results/run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1313_z_tracks_masks.h5']


In [143]:
tracks_fn = tracks_fn_results[0]

#### Now load the segmentation and z-tracks

In [129]:
with btrack.io.HDF5FileHandler(tracks_fn, 'r', obj_type='obj_type_1') as reader:
    segmentation = reader.segmentation
    tracks = [t for t in reader.tracks if len(t) >= 3]
    objects = reader.objects

[INFO][2023/08/22 01:07:23 pm] Opening HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/results/run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1313_z_tracks_masks.h5...
[INFO][2023/08/22 01:07:24 pm] Loading segmentation (4, 2304, 2304)
[INFO][2023/08/22 01:07:24 pm] Loading tracks/obj_type_1
[INFO][2023/08/22 01:07:24 pm] Loading LBEP/obj_type_1
[INFO][2023/08/22 01:07:24 pm] Loading objects/obj_type_1 (12, 5) (12 filtered: None)
[INFO][2023/08/22 01:07:24 pm] Loading objects/obj_type_1 (12, 5) (12 filtered: None)
[INFO][2023/08/22 01:07:24 pm] Closing HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/results/run1_23-01-001_23-01-005/2h pi/20230707_40X_23-01-001A3_Multichannel Z-Stack_20230707_1313_z_tracks_masks.h5


In [89]:
napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim = 2)

In [90]:
recolored_segmentation = btrack.utils.update_segmentation(segmentation, tracks, color_by='ID')

In [91]:
viewer = napari.Viewer(title = image_fn)

viewer.add_image(image, channel_axis = -1)
viewer.add_labels(segmentation, visible = False)
viewer.add_labels(recolored_segmentation)
viewer.add_tracks(napari_tracks)

<Tracks layer 'napari_tracks' at 0x7f1841744dc0>

In [92]:
print()

# Retracking

In [95]:
props = ('axis_major_length', # first define some cell properties 
         'axis_minor_length', # this helps improve the tracking
         'eccentricity',      # by comparing similarities between slices/frames
         'area', 
         'orientation',
         'mean_intensity', 
        )
objects = btrack.utils.segmentation_to_objects(
            segmentation = segmentation, # set the masks here 
            intensity_image = image, # provide the image so that the mean intensity can be measured
            properties = props, # provide the cell properties to improve tracker 
            use_weighted_centroid = False, 
            assign_class_ID=False,
            )

[INFO][2023/08/22 11:31:17 am] Localizing objects from segmentation...
100%|█████████████████████████| 61/61 [01:11<00:00,  1.18s/it]
[INFO][2023/08/22 11:32:29 am] Objects are of type: <class 'dict'>
[INFO][2023/08/22 11:32:31 am] ...Found 172246 objects in 61 frames.


In [96]:
with btrack.BayesianTracker() as tracker:
        # configure the tracker using a config file
        tracker.configure('/home/dayn/analysis/models/loc.json')
        ### set max search radius to a very limited radius 
        tracker.max_search_radius = 5
        # define tracking method
        tracker.tracking_updates = ["MOTION", "VISUAL"]
        # use visual features to track
        tracker.features = props
        # append the objects to be tracked
        tracker.append(objects)
        # set the volume
        tracker.volume=((0, segmentation.shape[1]), (0, segmentation.shape[2]), (-1e5, 1e5))
        # track them (in interactive mode)
        tracker.track(step_size=10)
        # generate hypotheses and run the global optimizer
        tracker.optimize()
        # get the tracks as a python list
        tracks = tracker.tracks

[INFO][2023/08/22 11:34:26 am] Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
[INFO][2023/08/22 11:34:26 am] Starting BayesianTracker session
[INFO][2023/08/22 11:34:26 am] Loading configuration file: /home/dayn/analysis/models/loc.json
[INFO][2023/08/22 11:34:26 am] Objects are of type: <class 'list'>
[INFO][2023/08/22 11:34:30 am] Starting tracking... 
[INFO][2023/08/22 11:34:30 am] Update using: ['VISUAL', 'MOTION']
[INFO][2023/08/22 11:35:26 am] Tracking objects in frames 0 to 9 (of 61)...
[INFO][2023/08/22 12:00:02 pm]  - Timing (Bayesian updates: 43739.25ms, Linking: 94.67ms)
[INFO][2023/08/22 12:00:02 pm]  - Probabilities (Link: 1.00000, Lost: 0.99998)
[INFO][2023/08/22 12:00:02 pm]  - Stats (Active: 5540, Lost: 687, Conflicts resolved: 17401)
[INFO][2023/08/22 12:00:02 pm] Tracking objects in frames 10 to 19 (of 61)...
[INFO][2023/08/22 12:13:11 pm]  - Timing (Bayesian updates: 38414.20ms, Linking: 76.59ms)
[INFO][2023/08/22 12:13:11 pm]  - Probabilities (L

GLPK Integer Optimizer 5.0
87044 rows, 65673 columns, 87824 non-zeros
65673 integer variables, all of which are binary
Preprocessing...
43522 rows, 65673 columns, 87824 non-zeros
65673 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 43522
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
43522 rows, 65673 columns, 87824 non-zeros
*     0: obj =   2.370127093e+05 inf =   0.000e+00 (16782)
*  4325: obj =   1.901104049e+05 inf =   0.000e+00 (12483)
Perturbing LP to avoid stalling [6925]...
*  8993: obj =   1.433704515e+05 inf =   0.000e+00 (7836)
* 13596: obj =   1.196027052e+05 inf =   0.000e+00 (3326)
Removing LP perturbation [17001]...
* 17001: obj =   1.084785872e+05 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+ 17001: mip =     not found yet >=   

[INFO][2023/08/22 12:38:04 pm] Ending BayesianTracker session


In [13]:
napari_tracks, _, _ = btrack.utils.tracks_to_napari(tracks, ndim = 2)

In [14]:
recolored_segmentation = btrack.utils.update_segmentation(segmentation, tracks, color_by='ID')

In [16]:
viewer = napari.Viewer(title = fn +'retracked')

# viewer.add_image(image, channel_axis = -1)
viewer.add_labels(segmentation, visible = False)
viewer.add_labels(recolored_segmentation)
viewer.add_tracks(napari_tracks)

<Tracks layer 'napari_tracks' at 0x7f57c78724f0>

# Retrack all 

In [2]:
start_directory = '/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/'
image_fns = find_tif_files(start_directory)
len(image_fns)

Searching for .tif files: 0it [00:00, ?it/s]

49

In [11]:
props = ('axis_major_length',
 'axis_minor_length',
 'eccentricity',
 'area',
 'orientation',
 'mean_intensity')

In [22]:
from macrohet import notify

In [23]:
for fn in tqdm(image_fns, total = len(image_fns)):
    # image = io.imread(fn)
    image_basename = os.path.basename(fn)
    tracks_basename = image_basename.replace('.tif','_z_tracks_masks.h5') #'20x_21-12-029B_A12346_Multichannel Z-Stack_20220819_295_z_tracks_masks.h5'
    tracks_fn_results = find_files_with_basename(tracks_basename, start_directory)
    if len(tracks_fn_results) > 1:
        raise ValueError("Found more than one matching file. Please ensure only one target file exists or refine your search criteria.")
    tracks_fn = tracks_fn_results[0]
    with btrack.io.HDF5FileHandler(tracks_fn, 'r', obj_type='obj_type_1') as reader:
        objects = reader.objects
        segmentation = reader.segmentation

    with btrack.BayesianTracker() as tracker:
        # configure the tracker using a config file
        tracker.configure('/home/dayn/analysis/models/loc.json')
        ### set max search radius to a very limited radius 
        tracker.max_search_radius = 5
        # define tracking method
        tracker.tracking_updates = ["MOTION", "VISUAL"]
        # use visual features to track
        tracker.features = props
        # append the objects to be tracked
        tracker.append(objects)
        # set the volume
        tracker.volume=((0, segmentation.shape[1]), (0, segmentation.shape[2]), (-1e5, 1e5))
        # track them (in interactive mode)
        tracker.track(step_size=10)
        # generate hypotheses and run the global optimizer
        tracker.optimize()
        # get the tracks as a python list
        tracks = tracker.tracks

    # save out alongside new images
    new_tracks_fn = fn.replace('.tif', '_sc_data.h5').replace('/run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/', '/home/dayn/data/homuncu_loc_temp/results/')    
    os.makedirs(os.path.dirname(new_tracks_fn), exist_ok=True)
    # save out 
    with btrack.io.HDF5FileHandler(new_tracks_fn, 
                                       'w', 
                                       obj_type='obj_type_1'
                                       ) as writer:
            writer.write_tracks(tracks)
            writer.write_segmentation(segmentation)
    # notify me
    notify.send_sms(f'{image_basename} complete')

  0%|          | 0/49 [00:00<?, ?it/s]

Searching start directory for filename: 0it [00:00, ?it/s]

[INFO][2023/08/22 03:00:47 pm] Opening HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/results/run0/DAPI-SPC-PDPN-ZO1/Day14_breath/20x_21-12-029B_A12346_Multichannel Z-Stack_20220819_295_z_tracks_masks.h5...
[INFO][2023/08/22 03:00:48 pm] Loading objects/obj_type_1 (7245, 5) (7245 filtered: None)
[INFO][2023/08/22 03:00:54 pm] Loading segmentation (81, 2304, 2304)
[INFO][2023/08/22 03:00:54 pm] Closing HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/results/run0/DAPI-SPC-PDPN-ZO1/Day14_breath/20x_21-12-029B_A12346_Multichannel Z-Stack_20220819_295_z_tracks_masks.h5
[INFO][2023/08/22 03:00:54 pm] Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
[INFO][2023/08/22 03:00:54 pm] Starting BayesianTracker session
[INFO][2023/08/22 03:00:54 pm] Loading configuration file: /home/dayn/analysis/models

GLPK Integer Optimizer 5.0
14668 rows, 11040 columns, 14746 non-zeros
11040 integer variables, all of which are binary
Preprocessing...
7334 rows, 11040 columns, 14746 non-zeros
11040 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 7334
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
7334 rows, 11040 columns, 14746 non-zeros
*     0: obj =   4.462087957e+04 inf =   0.000e+00 (3478)
*  3500: obj =   1.381541626e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+  3500: mip =     not found yet >=              -inf        (1; 0)
+  3500: >>>>>   1.381541626e+04 >=   1.381541626e+04   0.0% (1; 0)
+  3500: mip =   1.381541626e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMAL SOLUTION FOUND


[INFO][2023/08/22 03:01:02 pm] Writing objects/obj_type_1
[INFO][2023/08/22 03:01:02 pm] Writing labels/obj_type_1
[INFO][2023/08/22 03:01:02 pm] Loading objects/obj_type_1 (5932, 5) (5932 filtered: None)
[INFO][2023/08/22 03:01:02 pm] Writing properties/obj_type_1/orientation (5932,)
[INFO][2023/08/22 03:01:02 pm] Writing properties/obj_type_1/axis_major_length (5932,)
[INFO][2023/08/22 03:01:02 pm] Writing properties/obj_type_1/mean_intensity (5932, 4)
[INFO][2023/08/22 03:01:02 pm] Writing properties/obj_type_1/area (5932,)
[INFO][2023/08/22 03:01:02 pm] Writing properties/obj_type_1/axis_minor_length (5932,)
[INFO][2023/08/22 03:01:02 pm] Writing properties/obj_type_1/eccentricity (5932,)
[INFO][2023/08/22 03:01:02 pm] Writing tracks/obj_type_1
[INFO][2023/08/22 03:01:02 pm] Writing dummies/obj_type_1
[INFO][2023/08/22 03:01:02 pm] Writing LBEP/obj_type_1
[INFO][2023/08/22 03:01:02 pm] Writing fates/obj_type_1
[INFO][2023/08/22 03:01:06 pm] Closing HDF file: /home/dayn/data/homuncu

...

Searching start directory for filename: 0it [00:00, ?it/s]

[INFO][2023/08/22 06:09:52 pm] Opening HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/results_(old_organisation, currently reviewing)/run0/DAPI-NKX21-PDPN-ZO1/Day14_breath/20x_21-12-029B_A12346_Multichannel Z-Stack_20220819_286_z_tracks_masks.h5...
[INFO][2023/08/22 06:09:53 pm] Loading objects/obj_type_1 (38518, 5) (38518 filtered: None)
[INFO][2023/08/22 06:09:59 pm] Loading segmentation (81, 2304, 2304)
[INFO][2023/08/22 06:09:59 pm] Closing HDF file: /run/user/30046150/gvfs/smb-share:server=data2.thecrick.org,share=lab-gutierrezm/home/shared/Lung on Chip/image analysis_Nathan/results_(old_organisation, currently reviewing)/run0/DAPI-NKX21-PDPN-ZO1/Day14_breath/20x_21-12-029B_A12346_Multichannel Z-Stack_20220819_286_z_tracks_masks.h5
[INFO][2023/08/22 06:09:59 pm] Loaded btrack: /home/dayn/analysis/btrack/btrack/libs/libtracker.so
[INFO][2023/08/22 06:09:59 pm] Starting BayesianTracker session

GLPK Integer Optimizer 5.0
54976 rows, 41308 columns, 55128 non-zeros
41308 integer variables, all of which are binary
Preprocessing...
27488 rows, 41308 columns, 55128 non-zeros
41308 integer variables, all of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 27488
Solving LP relaxation...
GLPK Simplex Optimizer 5.0
27488 rows, 41308 columns, 55128 non-zeros
*     0: obj =   1.763870297e+05 inf =   0.000e+00 (12016)
*  8453: obj =   8.317013772e+04 inf =   0.000e+00 (3593)
* 12074: obj =   5.852145444e+04 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Integer optimization begins...
Long-step dual simplex will be used
+ 12074: mip =     not found yet >=              -inf        (1; 0)
+ 12074: >>>>>   5.852145444e+04 >=   5.852145444e+04   0.0% (1; 0)
+ 12074: mip =   5.852145444e+04 >=     tree is empty   0.0% (0; 1)
INTEGER OPTIMAL SOLUTION FOUN

[INFO][2023/08/22 06:14:00 pm]  - Fates.FALSE_POSITIVE: 11928 (of 13744)
[INFO][2023/08/22 06:14:00 pm]  - Fates.LINK: 10 (of 76)
[INFO][2023/08/22 06:14:00 pm]  - Fates.INITIALIZE_BORDER: 133 (of 493)
[INFO][2023/08/22 06:14:00 pm]  - Fates.INITIALIZE_LAZY: 1673 (of 13251)
[INFO][2023/08/22 06:14:00 pm]  - Fates.TERMINATE_BORDER: 122 (of 461)
[INFO][2023/08/22 06:14:00 pm]  - Fates.TERMINATE_BACK: 178 (of 787)
[INFO][2023/08/22 06:14:00 pm]  - Fates.TERMINATE_LAZY: 1506 (of 12496)
[INFO][2023/08/22 06:14:00 pm]  - TOTAL: 41308 hypotheses
[INFO][2023/08/22 06:14:00 pm] Completed optimization with 13734 tracks
[INFO][2023/08/22 06:14:01 pm] Ending BayesianTracker session
[INFO][2023/08/22 06:14:01 pm] Opening HDF file: /home/dayn/data/homuncu_loc_temp/results/image analysis_Nathan/Job_1/DAPI-NKX21-PDPN-ZO1/Day14_breath/20x_21-12-029B_A12346_Multichannel Z-Stack_20220819_286_sc_data.h5...
[INFO][2023/08/22 06:14:01 pm] Writing objects/obj_type_1
[INFO][2023/08/22 06:14:01 pm] Writing lab

ConnectionError: HTTPSConnectionPool(host='api.twilio.com', port=443): Max retries exceeded with url: /2010-04-01/Accounts/AC5c24c3260ed6b1c8289de1897f4e5b39/Messages.json (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7f582ac741c0>: Failed to resolve 'api.twilio.com' ([Errno -3] Temporary failure in name resolution)"))